In [1]:
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import numpy as np
import math
import csv
import os
import codecs
from scipy.spatial import distance

In [2]:
def create_folder(dir, folder_name):
    output_dir = os.path.join(dir, folder_name)

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [3]:
def create_image_from_letter(font_path, size_1: int, size_letter: int, letter, dir_normal_letters_imgs, dir_negative_letters_imgs):
  image = Image.new('RGB', (size_1, size_1), color=(255, 255, 255))
  font = ImageFont.truetype(font_path, size=size_letter)
  
  # Создаем объект для рисования текста
  draw = ImageDraw.Draw(image)

  # Рисуем букву в центре изображения
  draw.text((size_1//2, size_1//2), letter, font=font, fill=(0, 0, 0), anchor="mm")
  image.save(os.path.join(dir_normal_letters_imgs, f'{letter}.bmp'))
  img = invert(image)
  img.save(os.path.join(dir_negative_letters_imgs, f'{letter}.bmp'))

In [4]:
def monochrome(input_array):
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array
  
def binarization_b1w0(input_array):
    height, width = input_array.shape[:2]
    input_array1 = monochrome(input_array)
    # print(np.shape(input_array)[:])
    output_array = np.zeros((height, width), int)

    for i in range(height):
        for j in range(width):
            if input_array1[i][j] <= 200:
                output_array[i][j] = 1
            else:
                output_array[i][j] = 0
    
    return output_array

def first_nonzero(arr):
    first_non0 = -1
    for i in range (np.shape(arr)[0]):
        if arr[i] > 0:
            first_non0 = i
            break

    return first_non0

def last_nonzero(arr):
    arr_reversed = np.flip(arr) 
    first_non0 = -1

    for i in range (np.shape(arr)[0]):
        if arr_reversed[i] > 0:
            first_non0 = np.shape(arr)[0] - i-1
            break
    return first_non0


In [5]:
def profile_y(input_array):
    output_array2 = input_array
    return {'y_axis' : np.arange(0, output_array2.shape[0]).astype(int), # разметка оси y, 0 тк первое значение размера это колво строк
            'values' : np.sum(output_array2, axis=1) # проекция на ось y
            }


def profile_x(input_array):
    output_array1 = input_array
    return {'values' : np.sum(output_array1, axis=0), # проекция на ось x
            'x_axis' : np.arange(0, output_array1.shape[1]).astype(int) # разметка оси x
            }


def edit_img(img):
    img_array = np.array(img)

    img_array_01 = binarization_b1w0(img_array)
    img_array_cut = img_array_01[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    return img_array_cut


def X_diagram(input_array):
    
    height, width = input_array.shape[:2]
    p = profile_x(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    
    for x in range(width):
        for y in range(height):
            if y+1 <= p['values'][x]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255

    new_image = Image.fromarray(output_array.astype(np.uint8))
    new_image.save('NEW0.png')
    return new_image

    
def Y_diagram(input_array):
    height, width = input_array.shape[:2]
    print(height, width)
    p = profile_y(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)
    print(output_array.shape[:2])
    for x in range(width):
        for y in range(height):
            if x+1 <= p['values'][y]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255

    new_image = Image.fromarray(output_array.astype(np.uint8))
    new_image.save('NEW1.png')
    return new_image


def edit_img2(letter,path, DIR_negative_letters_imgs):
    input_image = Image.open(os.path.join(DIR_negative_letters_imgs, f'{letter}.bmp'))
    img_array = np.array(input_image)

    outpt = create_folder(path, "edited")
    neg = create_folder(outpt, "neg")
    # print(neg)
    pos = create_folder(outpt, "pos")

    img_array_01 = binarization_b1w0(img_array)
    img_array_cut = img_array[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    new_image = Image.fromarray(img_array_cut.astype(np.uint8))
    new_image.save(os.path.join(neg, f'{letter}.bmp'))

    new_image1 = np.invert(img_array_cut)
    new_image2 = Image.fromarray(new_image1.astype(np.uint8))
    new_image2.save(os.path.join(pos, f'{letter}.bmp'))


def XY_diagram_join(input_image):
    image_array = np.array(input_image)
    image_array_mono = monochrome(image_array)
    aaa = binarization_b1w0(image_array_mono)

    y = Y_diagram(aaa)
    x = X_diagram(aaa)

    height, width = image_array_mono.shape[:2]

    X_array = np.array(x)
    Y_array = np.array(y)
    zero_array  = np.ones((height,width),  dtype=image_array_mono.dtype)*255

    print(np.shape(image_array_mono),
          np.shape(X_array),
          np.shape(Y_array),
          np.shape(zero_array)
          )
    final0 = np.vstack([Y_array, zero_array])
    final1 = np.vstack([image_array_mono, X_array])
    final = np.hstack([final1, final0])   

    new_image = Image.fromarray(final.astype(np.uint8))
    new_image.save('diagram.png')

def save_profiles_as_images(img, letter, path):
    x = profile_x(img)
    y = profile_y(img)

    for axis in x.items():
        plt.bar(x['x_axis'], x['values'], color='blue')
        plt.xlabel('Сумма значений')  # Подпись оси X
        plt.ylabel('Пиксели')  # Подпись оси Y
        plt.savefig(os.path.join( path, f'profile_{letter}_axis_x.png'))  # Сохранение диаграммы как изображения PNG
        plt.close()  # Очистка

    for axis in y.items():
        plt.barh(y['y_axis'], y['values'], color='blue') # Используйте профили для создания столбчатой диаграммы
        plt.xlabel('Сумма значений')  # Подпись оси X
        plt.ylabel('Пиксели')  # Подпись оси Y
        plt.savefig(os.path.join( path, f'profile_{letter}_axis_y.png'))  # Сохранение диаграммы как изображения PNG
        plt.close()  # Очистка

In [6]:
def get_characteristics(img_edited):

    # Вес (масса чёрного) каждой четверти изображения символа
    weight = img_edited.sum()

    # Удельный вес (вес, нормированный к четверти площади)
    rel_weight = weight / (img_edited.shape[0] * img_edited.shape[1])

    # Координаты центра тяжести
    x_avg = 0
    for x, column in enumerate(img_edited.T):
        x_avg += sum((x + 1) * column)
    x_avg = x_avg / weight

    y_avg = 0
    for y, row in enumerate(img_edited):
        y_avg += sum((y + 1) * row)
    y_avg = y_avg / weight
    
    # Нормированные координаты центра тяжести
    rel_x_avg = (x_avg - 1) / (img_edited.shape[1] - 1)
    rel_y_avg = (y_avg - 1) / (img_edited.shape[0] - 1)

    # Осевые моменты инерции по горизонтали и вертикали
    iner_x = 0
    for y, row in enumerate(img_edited):
        iner_x += sum(row*(y + 1 - y_avg)**2)

    iner_y = 0
    for x, column in enumerate(img_edited.T):
        iner_y += sum(column*(x + 1 - x_avg)** 2 )

    # Нормированные осевые моменты инерции
    rel_iner_x = iner_x / (img_edited.shape[0]** 2 * img_edited.shape[1]** 2)
    rel_iner_y = iner_y / (img_edited.shape[0]** 2 * img_edited.shape[1]** 2)

    return {
        'weight': weight,
        'rel_weight': rel_weight,
        'center': [x_avg, y_avg],
        'rel_center': [rel_x_avg, rel_y_avg],
        'inertia': [iner_x, iner_y],
        'rel_inertia': [rel_iner_x, rel_iner_y]
    }

In [7]:
def save_characteristics_to_csv(letter, characteristics_list, output_file):
    

    with open(output_file, 'a', newline='') as csvfile:
        fields = [ 'Вес_чёрного', 'Норм_вес_чёрного', 'Центр_массы', 'Нормированный_центр_массы', 'Моменты_инерции', 'Нормированные_моменты_инерции']
        
        csvwriter = csv.DictWriter(csvfile, fieldnames=fields)
        csvwriter.writeheader()
        # csvwriter.writerow(characteristics_list)

        center_mass_x, center_mass_y = characteristics_list['center']
        normalized_center_mass_x, normalized_center_mass_y = characteristics_list['rel_center']
        normalized_center_mass_x, normalized_center_mass_y = characteristics_list['rel_center']
        normalized_inertia_x, normalized_inertia_y = characteristics_list['rel_inertia']
        inertia_x, inertia_y = characteristics_list['inertia']

        csvwriter.writerow({
            # 'Буква': letter,
            'Вес_чёрного': f"{characteristics_list['weight']}",
            'Норм_вес_чёрного': f"{characteristics_list['rel_weight']}",
            'Центр_массы': f"({center_mass_x}, {center_mass_y})",
            'Нормированный_центр_массы': f"({normalized_center_mass_x}, {normalized_center_mass_y})",
            'Моменты_инерции': f"({inertia_x}, {inertia_y})",
            'Нормированные_моменты_инерции': f"({normalized_inertia_x}, {normalized_inertia_y})"
            })

In [8]:
def errosion(input_array):

    height, width = input_array.shape[:2]
    output_array = input_array.copy()
    for i in range (1, width-1):
        for j in range (1, height-1):
            if input_array[j][i] == 1 and (input_array[j][i+1] == 0 or input_array[j][i-1] == 0):
                output_array[j][i] = 0
            else:
                output_array[j][i] = input_array[j][i]

    return output_array

def segmentation(input_array1):

    height, width = input_array1.shape[:2]
    input_array = errosion(input_array1)
    p = profile_x(input_array)
    list_borders = {
        'left' : [],
        'right' : [],
        'up': [],
        'down' : []
    }

    for i in range (p['x_axis'].shape[0]-1) :  
        if (p['values'][i] > 0) and (p['values'][i+1] == 0):
            list_borders['right'].append(i)
        if (p['values'][i] ==  0) and (p['values'][i+1] > 0):
            list_borders['left'].append(i)

    if list_borders['left'][0]>list_borders['right'][0]:
        list_borders['left'].append(0)
    if list_borders['left'][-1]<list_borders['right'][-1]:
        list_borders['right'].append(width-1)

    list_borders['left'].sort()
    list_borders['right'].sort()

    for k in range (len(list_borders['left'])):

        if list_borders['left'][k]<list_borders['right'][k]:

            new = np.zeros((height, list_borders['right'][k]-list_borders['left'][k]), int)
            height, width = new.shape[:2]
            for i in range (width):
                for j in range (height):
                    new[j][i]= input_array[j][list_borders['left'][k]+i]
            py = profile_y(new)
            list_borders['up'].append(first_nonzero(py['values']))
            list_borders['down'].append(last_nonzero(py['values']))


    return list_borders

def draw_brders(input_image, list):

    img_array = np.array(input_image)
    height, width = img_array.shape[:2]


    for k in range (len(list['left'])):
        x1 = list['left'][k]
        x2 = list['right'][k]
        y1 = list['up'][k]
        y2 = list['down'][k]
        

        for i in range (width):
            for j in range  (height):
                if (j >= y1 and j <=y2 ) and (i >= x1 and i <= x2):
                    
                    if (j == y1 or j == y2 or i == x1 or i ==x2):
                        img_array[j][i][0] = 255
                        img_array[j][i][1] = 0
                        img_array[j][i][2] = 0


    new_image = Image.fromarray(img_array)
    new_image.save('killme.png')

In [9]:
def copmpare(letter, letters,characteristics_list, text_charcteristics, comparison_list ):
    
    for letter1 in letters:

        comparison_list[f'{letter}'][f'{letter1}'] = {
            'rel_weight':  1- abs(characteristics_list[f'{letter1}']['rel_weight'] - text_charcteristics['rel_weight']), 
            'rel_center':  1 - distance.euclidean(characteristics_list[f'{letter1}']['rel_center'], 
                                                            text_charcteristics['rel_center']),
            'rel_inertia': 1 - distance.euclidean(characteristics_list[f'{letter1}']['rel_inertia'],
                                                            text_charcteristics['rel_inertia']) 
        }
    
# dst = distance.euclidean(a, b)


In [ ]:
def main():

    letters = ['A','B','C','Ç','D','E','F',
               'G','Ğ','H','I','İ','J','K',
               'L','M','N','O','Ö','P','R',
               'S','Ş','T','U','Ü','V','Y','Z']
 
    PATH_font = "C:/Users/Milana/Documents/oavi/lab5/input/TTimesb.ttf"
    DIR_cwd = os.getcwd()
    DIR_output = create_folder(DIR_cwd, "output")
    DIR_hitst = create_folder(DIR_output, "hists")
    DIR_normal_letters_imgs = create_folder(DIR_output, "normal")
    DIR_negative_letters_imgs = create_folder(DIR_output, "negative")
    characteristics_list = {}

    for letter4 in letters:
        
        create_image_from_letter(PATH_font, 600, 500, letter4, DIR_normal_letters_imgs, DIR_negative_letters_imgs)

        PATH_image = os.path.join(DIR_negative_letters_imgs, f'{letter4}.bmp')
        image = Image.open(PATH_image)
        
        img_array_b0w1 = edit_img(image)


        characteristics_list[f'{letter4}'] = get_characteristics(img_array_b0w1)
    
    # input_image = Image.open(os.path.join(DIR_negative_letters_imgs, f'{letters[10]}.bmp'))
    # input_array = np.array(input_image)

    # input_array1 = edit_img(input_array)


    # print(characteristics_list['Z'], '/n',text_charcteristics, '/n')
    comparison_list={}

    for letter in letters:
        create_image_from_letter(PATH_font, 550, 450, letter, DIR_normal_letters_imgs, DIR_negative_letters_imgs)

        input_image = Image.open(os.path.join(DIR_negative_letters_imgs, f'{letter}.bmp'))
        input_array = np.array(input_image)
        input_array1 = edit_img(input_array)
        text_charcteristics = get_characteristics(input_array1)
        
        comparison_list[f'{letter}'] = {}  # Создаем пустой словарь для каждой буквы
        for letter1 in letters:
            comparison_list[f'{letter}'][f'{letter1}'] = {
                    'rel_weight': 1 - abs(characteristics_list[f'{letter1}']['rel_weight'] - text_charcteristics['rel_weight']),
                    'rel_center': 1 - distance.euclidean(characteristics_list[f'{letter1}']['rel_center'], text_charcteristics['rel_center']),
                    'rel_inertia': 1 - distance.euclidean(characteristics_list[f'{letter1}']['rel_inertia'], text_charcteristics['rel_inertia'])
            }
            print(letter, letter1, comparison_list[f'{letter}'][f'{letter1}'])

    result = {}
    for letter3 in letters:
        max_w = 0 
        max_c = 0
        max_i = 0

        mult = 0
        max_mult = ''

        letter_w = ''
        letter_c = ''
        letter_i = ''

        for letter0 in letters:
            result[f'letter3'] = {}

            if comparison_list[f'{letter3}'][f'{letter0}']['rel_weight'] > max_w:
                max_w = comparison_list[f'{letter3}'][f'{letter0}']['rel_weight']
                letter_w = letter0
            if comparison_list[f'{letter3}'][f'{letter0}']['rel_center'] > max_c:
                max_c = comparison_list[f'{letter3}'][f'{letter0}']['rel_center']
                letter_c = letter0
            if comparison_list[f'{letter3}'][f'{letter0}']['rel_inertia'] > max_i:
                max_i = comparison_list[f'{letter3}'][f'{letter0}']['rel_inertia']
                letter_i = letter0                           
            # comparison_list = []
            # print(letter1, letter_w, letter_c, letter_i)
            if comparison_list[f'{letter3}'][f'{letter0}']['rel_weight']*comparison_list[f'{letter3}'][f'{letter0}']['rel_center']*comparison_list[f'{letter3}'][f'{letter0}']['rel_inertia']>mult:
                mult =max(comparison_list[f'{letter3}'][f'{letter0}']['rel_weight']*comparison_list[f'{letter3}'][f'{letter0}']['rel_center']*comparison_list[f'{letter3}'][f'{letter0}']['rel_inertia'],mult)
                max_mult =  letter0 

        result[f'letter3'] = {
            'weight' : letter_w,
            'center' : letter_c,
            'inertia' : letter_i,
            'mult' : max_mult
        }

        # print(letter3, letter0,comparison_list[f'{letter3}'][f'{letter0}'])       
        print(letter3 , result[f'letter3']['mult'])
        

if __name__ == '__main__':
    main()